# Older code from when I was using the Northern/Southern distinction

In [ ]:
# Version 2: heatplot of percentage of observations that where in-cloud
# With temperature


# Set up the figure with Gridspec
fig = plt.figure(figsize=(20, 10))
gs = fig.add_gridspec(1, 3, wspace=0.1)  # 1 row, 3 columns

# Create the heatmap on the left (2/3 of the width)
ax_heatmap = fig.add_subplot(gs[:, :2])

#plot text size options
hs=15
ss=15


pl = sns.heatmap(dist_df, annot=True, fmt=".0f", cmap='Blues', ax=ax_heatmap, 
            cbar_kws={"label": "% cloud observations", "location" : "left"},
            linewidths=0.5, annot_kws={'size':ss}, vmin=0)

pl.set_xlabel('Latitude ($^\circ$N)',fontsize=ss)
pl.set_ylabel('Altitude (m)', fontsize=ss)
pl.axes.get_yaxis().set_visible(False) # only show x-axis, the other plot shows the y-axis

# colorbar options
cbar = pl.collections[0].colorbar  # Get the colorbar from the heatmap
cbar.ax.tick_params(labelsize=ss)  # Change the size of the colorbar ticks
cbar.set_label('% cloud observations', fontsize=ss)  # Change colorbar label size


# ticklabel adjustments
xlab = pl.get_xticklabels() 
pl.set_xticklabels(labels = xlab, size=ss)
ylab = pl.get_yticklabels()
for label in ylab:
    label.set_text(label.get_text().replace('.0', '')) # remove all '.0'
pl.set_yticklabels(labels = ylab, size = ss, rotation=45)

# add overlines
# Create overlay plot
ax_lines = fig.add_axes(pl.get_position(), frameon=False)
ax_lines.set_xlim(79,70)
ax_lines.set_ylim(0,4500)
yticks = np.linspace (0,4500,300)
ax_lines.set_yticks(altitude_bins)
ax_lines.axes.get_xaxis().set_visible(False) # only show the y-axis, the other plot shows the xaxis (this shows the altitudes better.)
ax_lines.set_ylabel('Altitude (m)', fontsize=ss)
ax_lines.tick_params(labelsize=ss)


# Plot the limit between the two regions
ax_lines.axvline(lat_bands[1], ls='--', c='k', lw=2)
ax_lines.text(lat_bands[1] + 4, 5100, 'Northern marine region', c='b', weight='bold', fontsize=hs)
ax_lines.text(lat_bands[1] - 0.6, 5100, 'Southern marine region', c='r', weight='bold', fontsize=hs)
ax_lines.text(lat_bands[1] + 0.75, 4800, f"{lat_bands[1]:.2f}°", weight='bold',c='k', fontsize=ss)

ax_temp = fig.add_subplot(gs[:,2])

# Temperature southern and northern marine
# -- southern marine: mean and std
# Select variables to plot
sel_vars = ['altitude_bin', 'T']

south_df = south_ds[sel_vars].to_dataframe().reset_index()
s_t_mean = south_df.groupby('altitude_bin')['T'].mean()
s_t_std = south_df.groupby('altitude_bin')['T'].std()

# -- northern marine: mean and std
north_df = north_ds[sel_vars].to_dataframe().reset_index()
n_t_mean = north_df.groupby('altitude_bin')['T'].mean()
n_t_std = north_df.groupby('altitude_bin')['T'].std()

# the comma is to get the first element of the list returned
nmn, = ax_temp.plot(n_t_mean, n_t_mean.index, label = 'Northern \n marine region \n mean', lw=2,color = 'blue')
nstd = ax_temp.fill_betweenx(n_t_mean.index, n_t_mean+n_t_std, n_t_mean-n_t_std, facecolor='blue', alpha=0.2, label='N.M.R. std')

smn, = ax_temp.plot(s_t_mean, s_t_mean.index, label = 'Southern \n marine region \n mean', lw=2,color = 'red')
sstd = ax_temp.fill_betweenx(s_t_mean.index, s_t_mean+s_t_std, s_t_mean-s_t_std, facecolor='red', alpha=0.2, label='S.M.R. std')


# Create to title legend
legend_handles = [nmn, nstd, smn, sstd]
legend_labels = ['Mean', 'Std', 'Mean', 'Std']

# Create a legend with titles and custom handles
from matplotlib.lines import Line2D

# Create custom legend handles for the two titles
title1 = Line2D([0], [0], color='none', label=' Northern \n marine region', lw=0)  # Empty handle for title
title2 = Line2D([0], [0], color='none', label=' Southern \n marine region', lw=0)  # Empty handle for title

# Combine the handles and labels
handles = [title1] + [legend_handles[0], legend_handles[1], title2, legend_handles[2], legend_handles[3]]
labels = [title1.get_label(),legend_labels[0], legend_labels[1], title2.get_label(), legend_labels[2], legend_labels[3]]

# Add legend to the plot
ax_temp.legend(handles=handles, labels=labels, loc='upper right', fontsize=ss)

ax_temp.tick_params(axis='both', which='major', labelsize=ss)
ax_temp.set_ylabel('Altitude (m)', fontsize = ss)
ax_temp.yaxis.set_label_position("right")
ax_temp.set_xlabel('Temperature ($^\circ$C)', fontsize = ss)
ax_temp.set_xlim([-35, 0])

ax_temp.yaxis.tick_right()
ax_temp.set_yticks(altitude_bins)
ax_temp.set_ylim([0,5100])

#plt.title(f'Percentage of in cloud observations per total obs per alt_bin vs. lat_bin \n \
#            Preselection: {pre_text}\n \
#            in-cloud threshold method: {th_method}, threshold(s): {th}')
#plt.title(f'Percentage of in cloud observations \n per total obs per alt_bin vs. lat_bin', fontsize=20)
#plt.savefig(save_path + f'Perc_incloud_alt_lat_{sample_rate}s_{th_method}{preopt}.png', bbox_inches='tight')

#print(save_path + f'Perc_incloud_alt_lat_{sample_rate}s_{th_method}{preopt}.png')

In [ ]:
# make the distribution plots per flight
# heatplot of percentage of observations that where in-cloud
f, axs = plt.subplots(3,3,figsize=(20, 20))
plt.subplots_adjust(hspace=0.5)

ids = np.unique(ds_incloud.islasid.values)

for islasid, ax in zip(ids, axs.ravel()):
    #filter the two main dataframes on the flight
    ds_incloud_flight = ds_incloud.where(ds_incloud['islasid']==islasid, drop = True)
    ds_relevant_flight = ds_relevant.where(ds_relevant['islasid']==islasid, drop = True)
    
    # create dataframe with the data to visualize in the separate plots:
    # lat_bins is the same as for the full dataset
    
    count_df = create_counts(ds_incloud_flight, 'lat', lat_bins) # counts for incloud selection
    count_rel_df = create_counts(ds_relevant_flight, 'lat', lat_bins) # counts for all observations
    # Calculating the percentage of total observations per category that is in cloud:
    dist_df = ((count_df/count_rel_df)*100)

    #plot text size options
    hs=15
    ss=10
    #i = 0
    #j = 0
    pl = sns.heatmap(dist_df, annot=True, fmt=".0f", cmap='Blues', ax=ax, 
                cbar_kws={"label": "% cloud observations"},
                linewidths=0.5, annot_kws={'size':ss}, vmin=0, vmax = 100)
    
    ax.set_title(f'{islasid} ({len(ds_incloud_flight.time)} incloud obs.)')
    
    pl.set_xlabel('Latitude ($^\circ$N)',fontsize=ss)
    pl.set_ylabel('Altitude (m)', fontsize=ss)
    pl.axes.get_yaxis().set_visible(False) # only show x-axis, the other plot shows the y-axis

    # add overlines
    # Create overlay plot
    ax_lines = f.add_axes(ax.get_position(), frameon=False)
    ax_lines.set_xlim(79,70)
    ax_lines.set_ylim(0,4500)
    yticks = np.linspace (0,4500,300)
    ax_lines.set_yticks(altitude_bins)
    ax_lines.axes.get_xaxis().set_visible(False) # only show the y-axis, the other plot shows the xaxis (this shows the altitudes better.)
    ax_lines.set_ylabel('Altitude (m)', fontsize=ss)
    ax_lines.tick_params(labelsize=ss)

    # Plot the limit between the two regions
    ax_lines.axvline(lat_bands[1], ls='--', c='k', lw=2)
    #ax_lines.text(lat_bands[1] + 4.8, 0, 'Northern marine region', c='b', rotation = 90, fontsize=hs)
    #ax_lines.text(lat_bands[1] - 4.1, 0, 'Southern marine region', c='r', rotation = 90, fontsize=hs)
    ax_lines.text(lat_bands[1] + 1.5, 3900, f"{lat_bands[1]:.2f}°",c='k', fontsize=ss)

    

plt.savefig(save_path + f'Perc_incloud_alt_lat_{sample_rate}s_{th_method}{preopt}_singleflights.png', bbox_inches='tight')